# SixTrackLib

### ... and GPU simulations of particle beams in synchrotrons

<br /><br />

##### a notebook talk by Adrian Oeftiger (GSI) and Martin Schwinzerl (CERN, Uni Graz)

AMD and GSI meeting on 2 November 2020. See also [rendered talk on github $\nearrow$](https://aoeftiger.github.io/AMD-GSI-SixTrackLib-11-2020/) besides [source in github repo $\nearrow$](https://github.com/aoeftiger/AMD-GSI-SixTrackLib-11-2020/).

<br />
<center>
    <small>(hit space to go to the next slide)</small>
</center>

# ... the origins ...

Legacy simulation code [`SixTrack` $\nearrow$](http://sixtrack.web.cern.ch/SixTrack/):

* used in volunteered computing network BOINC: 
[40TFLOPS, >170'000 registered users $\nearrow$](https://lhcathome.cern.ch/lhcathome/server_status.php)
* 72'000 lines of `Fortran77/90` code
* **identical** numerical results across OS and hardware platforms
* re-implemented from scratch as C-templated code `SixTrackLib`
* adding GPU support via `openCL` (and `CUDA`)

$\implies$ aim for both 1) distributed BOINC computing and 2) HPC on GPU clusters

<div style="margin: auto; width: 60%;">
    <img src="images/boinc.png" alt="BOINC logo" style="width: 30%; float:left; margin-top: 25px;" />
    <img src="images/lhcathome.png" alt="LHC@home logo" style="width: 40%; float:left; margin-left: 10%;" />
</div>

# ... the applications of `SixTrackLib` ...

## GSI: SIS100 Magnet Field Errors

* SIS100 heavy-ion synchrotron under construction at GSI
* new magnets (dipoles, quadrupoles) built, measurements of field quality

$\implies$ `SixTrackLib` helps to evaluate impact of field errors on beam dynamics (long-term beam storage for 160'000 revolutions): 

$\implies$ now 2 mins instead of 16 h / simulation
~

# ... the modelled physics ...

# Beam Model

3D particle motion $\leadsto$ 6 phase space coordinates:
$$\mathbb{X}=(\underbrace{x, x'\vphantom{y'}}_{horizontal}, \underbrace{y, y'}_{vertical}, \underbrace{z, \delta\vphantom{y'}}_{longitudinal})$$

A beam $=$ state of $N$ macro-particles $=$ $6N$ values of phase space coordinates

<center>
    <img src="images/bunch.png" alt="particle bunch sketch" style="width:30%;" />
</center>

# Accelerator Model

Symplectic non-linear 6D tracking through elements along accelerator ring:

<p style="width: 80%; margin: 25px auto;">
    <img src='images/sis100.jpg' alt='SIS100 layout' style='width: 30%; float: left; margin-top: 0;' />
    <img src="images/transverse-model_empty.png" alt="sketch of one-turn map for ring" style="width:30%; float: left; margin-left: 5%; margin-top: 0;" />
    <img src="images/tracking.png" alt="tracking sketch" style="width:30%; float: left; margin-left: 5%; margin-top: 0;" />
</p>

# ... the numerics ...

## Simulations Requirements:


* **accurate**: long-term evolution $\leadsto$ double precision
* **fast**: heavy number crunching $\leadsto$ high-performance computing (HPC)<br />(in particular GPU parallelisation)
* **flexible**: iterative development of accelerator models with frequent updates $\leadsto$ python (API)

## Pseudo Code

```python
for n in range(nturns):
    for el_type, el_params in elements:
        for p in particles: // parallel!
            p = track[el_type](el_params, p)
```

$\implies$ **embarrassingly parallel** simulations: particles treated separately

$\leadsto$ current implementation: single kernel with case-switch in `openCL`

| What                   || Typical numbers                                     |
| :--------------------- || ---------------------------------------------------:|
| simulation length      || $\mathcal{O}(10^4)$ to $\mathcal{O}(10^7)$ `nturns` |
| accelerator `elements` || $\mathcal{O}(10^3)$ to $\mathcal{O}(10^5)$          |
| element types ($10$ to $20$) || $20$ to $\mathcal{O}(10^5)$ arithmetic operations |
| element parameters     || $1$ to $50$ `el_params`                             |
| macro-`particles`      || $\mathcal{O}(10^4)$ to $\mathcal{O}(10^6)$          |

<!-- 
* simulation length: $\mathcal{O}(10^4)$ to $\mathcal{O}(10^7)$ `nturns`
* accelerator `elements` ($1$ to $50$ `el_params`): $\mathcal{O}(10^3)$ to $\mathcal{O}(10^5)$
* macro-`particles`: $\mathcal{O}(10^4)$ to $\mathcal{O}(10^6)$
* element `el_type`s ($10$ to $20$): $20$ to $\mathcal{O}(10^5)$ arithmetic operations
-->
<!--* particle-to-particle interaction: binning, FFT, convolution, particle-in-cell, Poisson solvers-->

# ... what we work on! ...

## Prospect

* AMD Radeon Instinct cards in GSI HPC cluster
* lots of LHC@home users with AMD gaming GPUs

## Issues to collaborate on with AMD

* detailed **profiling** on AMD hardware
* **register spilling** throttles throughput: parallelisation approach to be optimised?

## Current Benchmarks

<div style="width:80%; margin: 25px auto;">
    <img src="images/stl_full.png" alt="SIS100 full errors" style="width: 45%; float:left; margin-top: 0;" />
    <img src="images/stl_optimised.png" alt="SIS100 dipole errors" style="width: 45%; float:left; margin-left:10%; margin-top: 0;" />
</div>

| AMD Radeon Instinct MI50 | AMD Radeon VII | NVIDIA RTX 2080 | NVIDIA V100 | CPU: AMD EPYC 7551 |
| ------------- | ---------- | --------------- | ----------- | ------------- |
| 6.7 TFLOPS | 3.5 TFLOPS | 0.44 TFLOPS | 7 TFLOPS | 0.65 TFLOPS |

# // Appendix: SixTrackLib Example //

In [ ]:
import sixtracklib

## MAD-X part first

In [ ]:
madx = Madx()
madx.options.echo = False
madx.options.warn = False
madx.options.info = False

Define SIS100 lattice in MAD-X:

In [ ]:
madx.call('./SIS100.seq')

Define SIS100 U${}^{28+}$ beam in MAD-X:

In [ ]:
from sixtracklib_setup import *

print ('mass = {:d}amu, charge = {:d}e, Ekin = {:.0f}MeV/u'.format(
    A, Q, 1e-6 * Ekin_per_nucleon))

In [ ]:
madx.command.beam(particle='ion', mass=A*nmass, charge=Q, energy=Etot);

In [ ]:
madx.input('''
            kqd := -0.2798446835;
            kqf := 0.2809756135;

            K1NL_S00QD1D :=  kqd ;
            K1NL_S00QD1F :=  kqf ;
            K1NL_S00QD2F :=  kqf ;
            K1NL_S52QD11 :=  1.0139780 * kqd ;
            K1NL_S52QD12 :=  1.0384325 * kqf ;
        ''');

In [ ]:
madx.use(sequence='sis100ring')

assert madx.command.select(
    flag='MAKETHIN',
    class_='QUADRUPOLE',
    slice_='9',
)

assert madx.command.select(
    flag='MAKETHIN',
    class_='SBEND',
    slice_='9',
)

assert madx.command.makethin(
    makedipedge=True,
    style='teapot',
    sequence='SIS100RING',
)

Match tunes to $Q_x=18.84$, $Q_y=18.73$:

In [ ]:
madx.use(sequence='sis100ring')

madx.input('''
match, sequence=SIS100RING;
global, sequence=SIS100RING, q1=18.84, q2=18.73;
vary, name=kqf, step=0.00001;
vary, name=kqd, step=0.00001;
lmdif, calls=500, tolerance=1.0e-10;
endmatch;
''');

In [ ]:
twiss = madx.twiss();

print ('\nQ1 = {:.2f} and Q2 = {:.2f}'.format(
    twiss.summary['q1'], twiss.summary['q2']))

## Now transfer SIS100 lattice to `PySixTrack`

In [ ]:
pysixtrack_elements = pysixtrack.Line.from_madx_sequence(
    madx.sequence.sis100ring, 
    exact_drift=True, install_apertures=False,
)

`pysixtrack_elements.elements` is a python list, we can play with it...

## Finally go to `SixTrackLib`

Transfer lattice from `PySixTrack`:

In [ ]:
elements = stl.Elements.from_line(pysixtrack_elements)

nturns = 2**16
elements.BeamMonitor(num_stores=nturns);

And define particle set:

In [ ]:
npart = 1
particles = stl.Particles.from_ref(npart, p0c=p0c)

In [ ]:
particles.x += 1e-6
particles.y += 1e-6

The trackjob will define the simulation kernel for us:

In [ ]:
job = stl.TrackJob(elements, particles, device=None)

$\leadsto$ note the `device` argument:

this can be `'opencl:0.0'` for openCL parallelisation, e.g. for multi-core CPU or GPU!

### Let's track to confirm the tunes from `MAD-X`:

In [ ]:
job.track_until(nturns)
job.collect()

# Evaluation:

In [ ]:
rec_x = job.output.particles[0].x
rec_y = job.output.particles[0].y

In [ ]:
plt.plot(rec_x[:128])
plt.plot(rec_y[:128])

In [ ]:
from sixtracklib_setup import plot_tunes_from_stl_results

plot_tunes_from_stl_results(twiss, job)